In [44]:
# Import libraries
# Main libraries 
import numpy as np
import pandas as pd

#Outliers
from scipy import stats
from dfcols import all_square_cols

# Balancing Libraries
import imblearn
from imblearn.over_sampling import SMOTE

#Models
from sklearn.ensemble import RandomForestClassifier

#Testing
from sklearn.metrics import f1_score

In [45]:
# Read files to pandas dataframes
train = pd.read_csv('data/Train.csv')
test = pd.read_csv('data/Test.csv')
sample_submission = pd.read_csv('data/SampleSubmission.csv')

In [46]:
#Split train-test
Xtr = train.head(8000)
Xte = train.tail(2863)
test_in = Xte.drop(columns=['Sample_ID','Label'])
test_res = Xte[["Label"]]

In [47]:
#Feature Transformation

In [48]:
#Outlier 
# Mean replacement of outliers
outlierAttributes = all_square_cols("elevation") + all_square_cols("slope") + all_square_cols("placurv") + all_square_cols("procurv") + all_square_cols("lsfactor")
Xtr = Xtr.copy(deep=True)
for column in Xtr:
    if (column in outlierAttributes):
        curcol = train[column]
        mean = curcol[np.abs(stats.zscore(curcol)) < 3].mean()
        Xtr.loc[np.abs(stats.zscore(curcol)) >= 3, column] = np.nan
        Xtr.fillna(mean, inplace=True)

In [49]:
#Balacing
oversample = SMOTE()
XtB, ytB = oversample.fit_resample(Xtr.drop(columns=['Sample_ID','Label']), Xtr[['Label']])

In [50]:
#Model Training + prediction
clf = RandomForestClassifier().fit(XtB,ytB)
y_pred = clf.predict(test_in)
f1 = f1_score(y_pred,test_res)
print(f1)

/var/folders/sd/r6t7q4t94y9gwkp7d5f63czh0000gn/T/ipykernel_73312/1390081265.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf = RandomForestClassifier().fit(XtB,ytB)


0.6535764375876578


# Test results
Splt Feature-Transformation Outlier-Method Balancing Model Training Score
70:30 None Mean-Method SMOTE RandomForests 0.6535
70:30 None None SMOTE RandomForests 0.6535